In [1]:
import json 
import numpy as np 
import cv2 
import os 
import time

In [ ]:
path_annotation = "/home/asilla/duongnh/datasets/instances_train2017.json"

In [11]:
file_annotation = open(path_annotation,'r')
json_data = json.load(file_annotation)

In [14]:
l_info = json_data['info']
l_image = json_data['images']
l_annotations = json_data['annotations']
l_categories = json_data['categories']

In [15]:
dict_image_id2image_name = {}
for info_image in l_image:
    file_name = info_image['file_name']
    image_id = info_image["id"]
    dict_image_id2image_name[image_id] = file_name

In [16]:
print(l_image[0])

{'license': 3, 'file_name': '000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}


In [17]:
print(len(l_annotations))

860001


In [29]:
imageid2imagename = {}
for info_image in l_image:
    file_name = info_image["file_name"]
    image_id = info_image["id"]
    imageid2imagename[image_id] = file_name

In [53]:
#Analysis Person, Motorbike annotation


def count_person_overlap_motor_bike(l_annotations):
    '''
    Loop annotation
    
    count person 
    count_motorbike
    create dict image id with {person_bbox: [], motorbike_bbox: []}
    loop all key 
    if person > 0, motorbike > 0: 
    loop person 
        loop mortorbike: -> check IOU: 50% -> l_image_id append -> count += 1
    '''
    dict_image_id = {}
    count_person_bbox = 0
    count_motor_bike = 0
    object_id_person = 1
    object_id_motorbike = 4
    for annotation in l_annotations:
        object_id = annotation['category_id']
        image_id = annotation['image_id']
        bbox = annotation['bbox']
        # keypoint_object = annotation['']
        if object_id == object_id_motorbike:
            count_motor_bike += 1
            if image_id not in dict_image_id:
                dict_image_id[image_id] = {}
                dict_image_id[image_id]['person'] = []
                dict_image_id[image_id]['mortorbike'] = []
                dict_image_id[image_id]['mortorbike'].append(bbox)
            else:
                dict_image_id[image_id]['mortorbike'].append(bbox)
        elif object_id == object_id_person:
            count_person_bbox += 1
            if image_id not in dict_image_id:
                dict_image_id[image_id] = {}
                dict_image_id[image_id]['person'] = []
                dict_image_id[image_id]['mortorbike'] = []
                dict_image_id[image_id]['person'].append(bbox)
            else:
                dict_image_id[image_id]['person'].append(bbox)
    print(count_motor_bike)
    print(count_person_bbox)
    return dict_image_id
            
    
    
    

In [54]:
dict_image_id_motorbike = count_person_overlap_motor_bike(l_annotations)

8725
262465


In [55]:
def bb_intersection_over_union(boxA, boxB):
    	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [56]:
def cal_IOU(bbox1, bbox2):
    '''
    Args: bbox: (xtop, ytop, w, h)
    Returns: IOU
    '''
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    boxA = (int(x1), int(y1), int(x1 + w1), int(y1 + h1))
    boxB = (int(x2), int(y2), int(x2 + w2), int(y2 + h2))
    
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [65]:
l_image_have_both = []
l_image_overlap_object = []
count_object_overlap  = 0

l_image_id = []
for image_id in dict_image_id_motorbike.keys():
    item_tmp = dict_image_id_motorbike[image_id]
    l_bbox_motorbike =item_tmp['mortorbike']
    l_bbox_person = item_tmp['person']
    iou_per_person = [0 for i in l_bbox_person]
    status_found = False
    if len(l_bbox_motorbike) > 0 and len(l_bbox_person) > 0:
        l_image_have_both.append(image_id)
        for index, bbox_person in enumerate(l_bbox_person):
            for bbox_motorbike in l_bbox_motorbike:
                iou_object = cal_IOU(bbox_person, bbox_motorbike)
                if  iou_object >= 0.05:
                    iou_per_person[index] = iou_object
                    count_object_overlap += 1
                    status_found = True
                    # break
    if status_found:
        l_image_overlap_object.append(image_id)


            

682


In [ ]:
#Visualize

In [96]:
import os
import cv2

In [84]:
path_data = "/home/asilla/duongnh/datasets/train2017"
l_filename = os.listdir(path_data)

In [85]:
print(len(l_filename))

118287


In [127]:

def visualize_bbox(image, bbox,color, iou, visualize_iou):
    xtop, ytop, width, height = bbox
    image = cv2.rectangle(image, (int(xtop), int(ytop)),\
                            (int(xtop + width), int(ytop + height)), color,1)
    if visualize_iou and iou > 0:
        font = cv2.FONT_HERSHEY_SIMPLEX
        color = (255, 255, 255)
        thickness = 2
        fontScale = 1
        iou = round(iou, 2)
        image = cv2.putText(image, str(iou), (int(xtop), int(ytop +height/2)),\
                             font, fontScale, color, thickness,\
                             cv2.LINE_AA)
    return image

In [128]:
print(len(l_image_id))

107


In [129]:
def visualize_keypoint(image, keypoints, color):
    global join_keypoint_categories
    l_keypoint = []
    for i in range(17):
        x = int(keypoints[i * 3])
        y = int(keypoints[i * 3 + 1])
        l_keypoint.append((x,y))
        if x != 0 and y != 0:
            # print(x,y)
            image = cv2.circle(image, (x,y), 2,(0,255,0), -1)
    for pair_point in join_keypoint_categories:
        start_point_index, end_point_index = pair_point
        if l_keypoint[start_point_index - 1][0] != 0 and l_keypoint[start_point_index - 1][1] != 0 and\
            l_keypoint[end_point_index - 1][0] != 0 and l_keypoint[end_point_index - 1][1] != 0:
            image = cv2.line(image, l_keypoint[start_point_index - 1], l_keypoint[end_point_index - 1], color, 2)
    return image 

In [130]:
l_color_keypointjoint = [(255,0,0), (0,255,255), (255,0,255),\
                        (0,140,255),(32,165,218),(0,128,128),\
                        (0,100,0),(79,79,47),(204,209,72),\
                        (112,25,25),(139,0,139)]

In [131]:
count_visualize = 0
# count_image_overlap = 0
for image_id in l_image_id:
    file_name = imageid2imagename[image_id]
    if image_id in dict_image_id_filename_person:
        file_name_person = dict_image_id_filename_person[image_id]
        if file_name_person == file_name:
            if file_name in l_filename:
                image = cv2.imread(path_data + "/" + file_name)
                l_keypoints = l_image_id_annotation_person[image_id]
                
                for index, keypoint in enumerate(l_keypoints):
                    tmp_index = index
                    if tmp_index > len(l_color_keypointjoint) - 1:
                        tmp_index = 0
                    color = l_color_keypointjoint[tmp_index]
                    image = visualize_keypoint(image, keypoint, color)

                annotation = dict_image_id_dog[image_id]
                l_iou_score =  annotation['iou_score']
                l_bbox_object = annotation["object"]
                l_bbox_person = annotation["person"]
                for bbox_object in l_bbox_object:
                    image = visualize_bbox(image, bbox_object,(0,0,255),0, False)
                for index, bbox_person in enumerate(l_bbox_person):
                    image = visualize_bbox(image, bbox_person,(0,255,0),l_iou_score[index], True)
                cv2.imwrite("Log_visualize_person_dog/" + file_name, image)
                count_visualize += 1
                print("Save")
                if count_visualize > 19:
                    break

Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save
Save


In [43]:
print(count_image_overlap)

591
